export dataset: to `ic2s2_datathon`

---

In [1]:
import os, helpers, warnings, tqdm
warnings.simplefilter('once')

In [2]:
import pandas as pd

In [28]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://zhiyzuo@localhost:5432/ic2s2_datathon')

---

loop through all excel files

In [4]:
base_dir = 'data/'

In [10]:
file_list = [base_dir+f for f in os.listdir(base_dir) if f.endswith('.xlsx')]
len(file_list)

85

use a for loop to create a giant user and text file

In [11]:
user_df = list()
text_df = list()

In [12]:
file_list[38]

'data/score_Bone infections.xlsx'

In [13]:
for f in tqdm.tqdm_notebook(file_list):
    #print(f)
    u_df, t_df = helpers.extract_user_text_data(f)
    user_df.append(u_df)
    text_df.append(t_df)
len(user_df) == len(text_df), len(text_df) == len(file_list)

/Users/zhiyzuo/.virtualenvs/py3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/zhiyzuo/OneDrive - University of Iowa/2018summer/2018IC2S2/datathon/helpers.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  question_df['type'] = 'q'


(True, True)

In [15]:
user_df = pd.concat(user_df,ignore_index=True)
user_df.shape

(52286, 5)

In [16]:
user_df.head(2)

,age,gender,role,user_id,id
0,68.0,Male,Medical Doctor,Dr.M.jagesh kamath,0
1,73.0,Male,Nurse Assistant,"John Kenyon, CNA",1


duplicates

In [17]:
user_df.drop_duplicates(subset=['user_id'], inplace=True, keep='first')
user_df.shape

(21127, 5)

In [19]:
text_df = pd.concat(text_df, ignore_index=True)
text_df.shape

(49392, 6)

In [20]:
text_df.head(2)

,qa_id,text,asker_id,answerer_id,type,disease
0,0,I normally had 120/80 blood pressure. While i...,macacat210,Dr.M.jagesh kamath,q,diseases of the aorta
1,1,Hello. I have developed hip and knee pain. It ...,mial7,"John Kenyon, CNA",q,diseases of the aorta


save to a database

In [29]:
user_df.to_sql('user', con=engine, index=False)
text_df.to_sql('text', con=engine, index=False)